In [1]:
import math
from math import log, sqrt, exp
import numpy as np
import scipy.stats as si
from scipy.stats import norm
from scipy.interpolate import interp1d
from scipy.integrate import quad

from scipy.optimize import brentq
from scipy.optimize import least_squares
from scipy.optimize import fmin
from scipy.integrate import quad

from matplotlib import pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons
import plotly.graph_objects as go

import pandas as pd
import datetime as dt
import warnings

In [2]:
# Settings the warnings to be ignored 
warnings.filterwarnings('ignore') 

%matplotlib ipympl

In [3]:
SPX_spot=3662.45
SPY_spot=366.02

In [4]:
# Use at the money strike to get option volatility

SPX_strike=3662.45
SPY_strike=366.02

In [5]:
Expiry_date = pd.to_datetime("20210115", format="%Y%m%d")

# Functions

In [6]:
def h_func(x):
    return x ** (1/3) + (3/2) * np.log(x) +10.0
def h_prime(x):
    return (1/3) * (x**(-2/3)) + (3/2) * (1/x)
def h_prime_prime(x):
    return (-2/9) * (x**(-5/3)) + (3/2) * (-1/(x**2))

In [7]:
def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)


def BlackScholesPut(S, K, r, sigma, T):
    return BlackScholesCall(S, K, r, sigma, T) - S + K*np.exp(-r*T)


In [8]:
def BS_impliedVolatility(S, K, r, price, T, payoff):
    try:
        if (payoff.lower() == 'call'):
            impliedVol = brentq(lambda x: price -
                                BlackScholesCall(S, K, r, x, T),
                                1e-12, 10.0)
        elif (payoff.lower() == 'put'):
            impliedVol = brentq(lambda x: price -
                                BlackScholesPut(S, K, r, x, T),
                                1e-12, 10.0)
        else:
            raise NameError('Payoff type not recognized')
    except Exception:
        impliedVol = np.nan

    return impliedVol

In [9]:
def BachelierCall(S:float, 
                  K:float, 
                  r:float, 
                  sigma:float, 
                  T:float):
    
    sigma = sigma*S
    
    # This is the last line in slide 7 where we define x_star
    x = (K-S)/(sigma*np.sqrt(T))
    
    # This is the last line in slide 8, remember to flip the sign of x
    value = \
        np.exp(-r*T)*((S-K)*norm.cdf(-x) 
                      + sigma*np.sqrt(T)*norm.pdf(-x))
    
    return value

In [10]:
def BachelierPut(S:float, 
                  K:float, 
                  r:float, 
                  sigma:float, 
                  T:float):
    
    sigma = sigma*S
    
    # This is the last line in slide 7 where we define x_star
    x = (K-S)/(sigma*np.sqrt(T))
    
    # This is the last line in slide 8, but it is now strike minus spot price and we no longer flip x
    value = \
        np.exp(-r*T)*((K-S)*norm.cdf(x) 
                      + sigma*np.sqrt(T)*norm.pdf(x))
    
    return value

In [11]:
def Bachelier_impliedVolatility(S, K, r, price, T, payoff):
    try:
        if (payoff.lower() == 'call'):
            impliedVol = brentq(lambda x: price -
                                BachelierCall(S, K, r, x, T),
                                1e-12, 10.0)
        elif (payoff.lower() == 'put'):
            impliedVol = brentq(lambda x: price -
                                BachelierPut(S, K, r, x, T),
                                1e-12, 10.0)
        else:
            raise NameError('Payoff type not recognized')
    except Exception:
        impliedVol = np.nan

    return impliedVol

# Data Cleaning

In [12]:
SPX_df = pd.read_csv("SPX_options.csv")
SPY_df = pd.read_csv("SPY_options.csv")
Rates_df = pd.read_csv("zero_rates_20201201.csv")

In [13]:
Rates_df["date"] = pd.to_datetime(Rates_df["date"], format="%Y%m%d")

In [14]:
Rates_df["rate_decimal"] = Rates_df["rate"] / 100

In [15]:
Rates_df = Rates_df.drop(["date"], axis=1)

Rates_df.set_index("days", inplace=True)

Rates_df = Rates_df.reindex(np.arange(Rates_df.index.min(), Rates_df.index.max() + 1))

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html
Rates_df = Rates_df.interpolate(method="linear")

In [16]:
Rates_df

,rate,rate_decimal
days,,
7,0.102280,0.001023
8,0.104255,0.001043
9,0.106229,0.001062
10,0.108204,0.001082
11,0.110179,0.001102
...,...,...
3572,0.955703,0.009557
3573,0.955906,0.009559
3574,0.956109,0.009561


### Prepare data for SPX

In [17]:
SPX_df

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style
0,20201201,20201218,C,100000,3547.6,3570.5,E
1,20201201,20201218,C,200000,3447.6,3470.5,E
2,20201201,20201218,C,300000,3347.7,3370.6,E
3,20201201,20201218,C,400000,3247.7,3270.6,E
4,20201201,20201218,C,500000,3147.7,3170.6,E
...,...,...,...,...,...,...,...
2067,20201201,20210219,P,5000000,1333.1,1350.5,E
2068,20201201,20210219,P,5100000,1431.8,1454.7,E
2069,20201201,20210219,P,5200000,1531.7,1554.6,E
2070,20201201,20210219,P,5300000,1631.5,1654.4,E


In [18]:
SPX_df["date"] = pd.to_datetime(SPX_df["date"], format="%Y%m%d")
SPX_df["exdate"] = pd.to_datetime(SPX_df["exdate"], format="%Y%m%d")

In [19]:
SPX_df["days_to_expiry"] = (SPX_df["exdate"] - SPX_df["date"]) / pd.Timedelta(days=1)
SPX_df["years_to_expiry"] = SPX_df["days_to_expiry"] / 365

In [20]:
SPX_df["mid_price"] = 0.5 * (SPX_df["best_bid"] + SPX_df["best_offer"])

In [21]:
SPX_df["strike_price"] = SPX_df["strike_price"] / 1000

In [22]:
SPX_df["options_type"] = SPX_df["cp_flag"].map(lambda x: "call" if x == "C" else "put")

In [23]:
SPX_df=SPX_df.merge(Rates_df,
                    left_on="days_to_expiry",
                    right_index=True)

In [24]:
# Filter the dataframe to get only exdates which we want

SPX_df = SPX_df [SPX_df ["exdate"] == Expiry_date].reset_index(drop=True)

In [25]:
# impl market volatility column
SPX_df["vols"] = SPX_df.apply(
    lambda x: BS_impliedVolatility(
        SPX_spot,
        x["strike_price"],
        x["rate_decimal"],
        x["mid_price"],
        x["years_to_expiry"],
        x["options_type"]
    ),
    axis=1,
)

# axis = (1,)
SPX_df.dropna(inplace=True)
SPX_df.reset_index(drop=True,inplace=True)

In [26]:
SPX_df

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,days_to_expiry,years_to_expiry,mid_price,options_type,rate,rate_decimal,vols
0,2020-12-01,2021-01-15,C,2570.0,1087.2,1099.6,E,45.0,0.123288,1093.40,call,0.205108,0.002051,0.361156
1,2020-12-01,2021-01-15,C,2575.0,1082.3,1094.6,E,45.0,0.123288,1088.45,call,0.205108,0.002051,0.364647
2,2020-12-01,2021-01-15,C,2580.0,1077.3,1089.6,E,45.0,0.123288,1083.45,call,0.205108,0.002051,0.362672
3,2020-12-01,2021-01-15,C,2590.0,1067.4,1079.7,E,45.0,0.123288,1073.55,call,0.205108,0.002051,0.368049
4,2020-12-01,2021-01-15,C,2610.0,1047.6,1059.9,E,45.0,0.123288,1053.75,call,0.205108,0.002051,0.374739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630,2020-12-01,2021-01-15,P,5000.0,1330.8,1346.6,E,45.0,0.123288,1338.70,put,0.205108,0.002051,0.391899
631,2020-12-01,2021-01-15,P,5100.0,1428.3,1451.2,E,45.0,0.123288,1439.75,put,0.205108,0.002051,0.434093
632,2020-12-01,2021-01-15,P,5200.0,1528.3,1551.2,E,45.0,0.123288,1539.75,put,0.205108,0.002051,0.455783
633,2020-12-01,2021-01-15,P,5300.0,1628.2,1651.1,E,45.0,0.123288,1639.65,put,0.205108,0.002051,0.475063


### Prepare data for SPY

In [27]:
SPY_df["date"] = pd.to_datetime(SPY_df["date"], format="%Y%m%d")
SPY_df["exdate"] = pd.to_datetime(SPY_df["exdate"], format="%Y%m%d")

In [28]:
SPY_df["days_to_expiry"] = (SPY_df["exdate"] - SPY_df["date"]) / pd.Timedelta(days=1)
SPY_df["years_to_expiry"] = SPY_df["days_to_expiry"] / 365

In [29]:
SPY_df["mid_price"] = 0.5 * (SPY_df["best_bid"] + SPY_df["best_offer"])

In [30]:
SPY_df["strike_price"] = SPY_df["strike_price"] / 1000

In [31]:
SPY_df["options_type"] = SPY_df["cp_flag"].map(lambda x: "call" if x == "C" else "put")

In [32]:
SPY_df=SPY_df.merge(Rates_df,
                    left_on="days_to_expiry",
                    right_index=True)

In [33]:
# Filter the dataframe to get only exdates which we want

SPY_df = SPY_df [SPY_df ["exdate"] == Expiry_date].reset_index(drop=True)

In [34]:
# impl market volatility column
SPY_df["vols"] = SPY_df.apply(
    lambda x: BS_impliedVolatility(
        SPY_spot,
        x["strike_price"],
        x["rate_decimal"],
        x["mid_price"],
        x["years_to_expiry"],
        x["options_type"]
    ),
    axis=1,
)
axis = (1,)
SPY_df.dropna(inplace=True)
SPY_df.reset_index(drop=True,inplace=True)

In [35]:
SPY_df

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,days_to_expiry,years_to_expiry,mid_price,options_type,rate,rate_decimal,vols
0,2020-12-01,2021-01-15,C,215.0,150.84,151.31,A,45.0,0.123288,151.075,call,0.205108,0.002051,0.397950
1,2020-12-01,2021-01-15,C,216.0,149.84,150.31,A,45.0,0.123288,150.075,call,0.205108,0.002051,0.383523
2,2020-12-01,2021-01-15,C,217.0,148.84,149.31,A,45.0,0.123288,149.075,call,0.205108,0.002051,0.359591
3,2020-12-01,2021-01-15,C,221.0,144.84,145.32,A,45.0,0.123288,145.080,call,0.205108,0.002051,0.426489
4,2020-12-01,2021-01-15,C,222.0,143.84,144.32,A,45.0,0.123288,144.080,call,0.205108,0.002051,0.420976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,2020-12-01,2021-01-15,P,535.0,170.18,170.65,A,45.0,0.123288,170.415,put,0.205108,0.002051,0.619266
479,2020-12-01,2021-01-15,P,540.0,175.18,175.65,A,45.0,0.123288,175.415,put,0.205108,0.002051,0.631142
480,2020-12-01,2021-01-15,P,545.0,180.18,180.65,A,45.0,0.123288,180.415,put,0.205108,0.002051,0.642859
481,2020-12-01,2021-01-15,P,550.0,185.18,185.65,A,45.0,0.123288,185.415,put,0.205108,0.002051,0.654424


### Getting the Black-Scholes sigma and Bachelier sigma for SPX

In [36]:
# Find price of ATM option
SPX_ATM = np.interp(SPX_strike, 
                    SPX_df["strike_price"], 
                    SPX_df["mid_price"])

In [37]:
SPX_BS_call_sigma = BS_impliedVolatility(SPX_spot,
                                       SPX_strike,
                                       SPX_df["rate_decimal"][0],
                                       SPX_ATM,
                                       SPX_df["years_to_expiry"][0],
                                       payoff = 'call')


SPX_BS_put_sigma = BS_impliedVolatility(SPX_spot,
                                      SPX_strike,
                                      SPX_df["rate_decimal"][0],
                                      SPX_ATM,
                                      SPX_df["years_to_expiry"][0],
                                      payoff = 'put')

SPX_ATM_sigma = (SPX_BS_call_sigma + SPX_BS_put_sigma)/2

print(f"ATM sigma for SPX using the Black-Scholes model is {SPX_ATM_sigma:.5f}")


ATM sigma for SPX using the Black-Scholes model is 0.18748


In [38]:
# Get bachelier sigma from the Black-Scholes price of option to get the Bachelier model volatility equivalent.

SPX_bachelier_sigma = (SPX_ATM * np.exp(SPX_df["rate_decimal"][0] * SPX_df["years_to_expiry"][0])) / (SPX_spot * np.sqrt(SPX_df["years_to_expiry"][0] / (2 * np.pi)))

print(f"ATM sigma for SPX using the Bachelier model is {SPX_bachelier_sigma :.5f}")

ATM sigma for SPX using the Bachelier model is 0.18747


In [39]:
SPX_Bachelier_call_sigma = Bachelier_impliedVolatility(SPX_spot,
                                       SPX_strike,
                                       SPX_df["rate_decimal"][0],
                                       SPX_ATM,
                                       SPX_df["years_to_expiry"][0],
                                       payoff = 'call')


SPX_Bachelier_put_sigma = Bachelier_impliedVolatility(SPX_spot,
                                      SPX_strike,
                                      SPX_df["rate_decimal"][0],
                                      SPX_ATM,
                                      SPX_df["years_to_expiry"][0],
                                      payoff = 'put')

SPX_Bachelier_sigma = (SPX_Bachelier_call_sigma + SPX_Bachelier_put_sigma)/2

print(f"ATM sigma for SPX using the Bachelier model is {SPX_Bachelier_sigma:.5f}")

ATM sigma for SPX using the Bachelier model is 0.18747


### Getting the Black-Scholes sigma and Bachelier sigma for SPY

In [40]:
# Find price of ATM option
SPY_ATM = np.interp(SPY_strike, 
                    SPY_df["strike_price"], 
                    SPY_df["mid_price"])

In [41]:
SPY_BS_call_sigma = BS_impliedVolatility(SPY_spot,
                                       SPY_strike,
                                       SPY_df["rate_decimal"][0],
                                       SPY_ATM,
                                       SPY_df["years_to_expiry"][0],
                                       payoff = 'call')


SPY_BS_put_sigma = BS_impliedVolatility(SPY_spot,
                                      SPY_strike,
                                      SPY_df["rate_decimal"][0],
                                      SPY_ATM,
                                      SPY_df["years_to_expiry"][0],
                                      payoff = 'put')

SPY_ATM_sigma = (SPY_BS_call_sigma + SPY_BS_put_sigma)/2

print(f"ATM sigma for SPY using the Black-Scholes model is {SPY_ATM_sigma:.5f}")

ATM sigma for SPY using the Black-Scholes model is 0.19684


In [42]:
# Get bachelier sigma from interpolated price of the ATM option

SPY_bachelier_sigma = (SPY_ATM * np.exp(SPY_df["rate_decimal"][0] * SPY_df["years_to_expiry"][0])) / (SPY_spot * np.sqrt(SPY_df["years_to_expiry"][0] / (2 * np.pi)))

print(f"ATM sigma for SPY using the Bachelier model is {SPY_bachelier_sigma :.5f}")

ATM sigma for SPY using the Bachelier model is 0.19682


In [43]:
SPY_Bachelier_call_sigma = Bachelier_impliedVolatility(SPY_spot,
                                       SPY_strike,
                                       SPY_df["rate_decimal"][0],
                                       SPY_ATM,
                                       SPY_df["years_to_expiry"][0],
                                       payoff = 'call')


SPY_Bachelier_put_sigma = Bachelier_impliedVolatility(SPY_spot,
                                      SPY_strike,
                                      SPY_df["rate_decimal"][0],
                                      SPY_ATM,
                                      SPY_df["years_to_expiry"][0],
                                      payoff = 'put')

SPY_Bachelier_sigma = (SPY_Bachelier_call_sigma + SPY_Bachelier_put_sigma)/2

print(f"ATM sigma for SPY using the Bachelier model is {SPY_Bachelier_sigma:.5f}")

ATM sigma for SPY using the Bachelier model is 0.19682


# Exotic No.1 : Black-Scholes

Expected Black Scholes payoff is defined as

$$
E[V_T]= {S_0}^\frac{1}{3}e^{\frac{rT}{3}}e^{\frac{-\sigma^2T}{9}} + 1.5(log{S_0} + (r-\frac{\sigma^2}{2})T) + 10
$$

Therefore, the price is

$$
V_0 = e^{-rT}E[V_T]
$$


#

In [44]:
# black scholes payoff
def bs_price(S, rate, sigma, T):
    return np.exp(-rate * T) * (
        (
            np.power(S, 1.0 / 3.0)
            * np.exp((rate - 0.5 * sigma**2) * T * (1.0 / 3.0))
            * np.exp(0.5 * (1.0 / 9.0) * T * sigma**2)
            + 1.5 * (np.log(S) + (rate - 0.5 * sigma**2) * T)
            + 10
        )
    )

In [45]:
SPX_BS_price = bs_price(SPX_spot, 
                        SPX_df["rate_decimal"][0], 
                        SPX_ATM_sigma, 
                        SPX_df["years_to_expiry"][0])

print(f"Price of the Exotic option using the Black-Scholes model with SPX as the underlying is {SPX_BS_price:5f}")

Price of the Exotic option using the Black-Scholes model with SPX as the underlying is 37.704607


In [46]:
SPY_BS_price = bs_price(SPY_spot, 
                        SPY_df["rate_decimal"][0], 
                        SPY_ATM_sigma, 
                        SPY_df["years_to_expiry"][0])

print(f"Price of the Exotic option using the Black-Scholes model with SPY as the underlying is {SPY_BS_price:5f}")

Price of the Exotic option using the Black-Scholes model with SPY as the underlying is 25.994282


# Exotic No.1 : Bachelier

Expected Bachelier payoff defined as

$$
E[V_T] = \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} (S_0 + \sigma S_0 \sqrt{T} x)^\frac{1}{3} e^\frac{-x^2}{2}\,dx +
\frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} 1.5log(S_0 + \sigma S_0 \sqrt{T} x) e^\frac{-x^2}{2}\,dx
+10
$$

Note $$S_T = S_0 + \sigma S_0W_T$$

Therefore, the price is

$$
V_0 = e^{-rT}E[V_T]
$$


In [47]:
def integrand_1(x, S, sigma, T):
    return (
        (1 / np.sqrt(2 * np.pi))
        * np.power((S + sigma * S * np.sqrt(T) * x), 1.0 / 3.0)
        * np.exp(-0.5 * np.power(x, 2))
    )

In [48]:
def integrand_2(x, S, sigma, T):
    return (
        (1 / np.sqrt(2 * np.pi))
        * 1.5
        * np.log(S + sigma * S * np.sqrt(T) * x)
        * np.exp(-0.5 * np.power(x, 2))
    )

In [49]:
def bachelier_price(S, rate, sigma, T):
    lower_bound = -1 / (sigma * np.sqrt(T))  # log term lower bound
    I_1 = quad(lambda x: integrand_1(x, S, sigma, T), lower_bound, np.inf)
    I_2 = quad(lambda x: integrand_2(x, S, sigma, T), lower_bound, np.inf)
    V_0_bachelier = np.exp(-rate * T) * (I_1[0] + I_2[0] + 10)
    return V_0_bachelier

In [50]:
SPX_Bachelier_price = bachelier_price(SPX_spot, 
                        SPX_df["rate_decimal"][0], 
                        SPX_ATM_sigma, 
                        SPX_df["years_to_expiry"][0])

print(f"Price of the Exotic option using the Bachelier model with SPX as the underlying is {SPX_Bachelier_price:5f}")

Price of the Exotic option using the Bachelier model with SPX as the underlying is 37.702870


In [51]:
SPY_Bachelier_price = bachelier_price(SPY_spot, 
                        SPY_df["rate_decimal"][0], 
                        SPY_ATM_sigma, 
                        SPY_df["years_to_expiry"][0])

print(f"Price of the Exotic option using the Bachelier model with SPY as the underlying is {SPY_Bachelier_price:5f}")

Price of the Exotic option using the Bachelier model with SPY as the underlying is 25.993253


# Exotic No.1 : SABR

For SABR payoff, we must retrieve the volatility using previous calibration result

$$
h(S_T) = S_T^{\frac{1}{3}} + 1.5 log(S_T) + 10 \\
h''(S_T) = -\frac{2}{9}S_T^{-\frac{5}{3}} - 1.5\frac{1}{S_T^2} \\
F = S_0 e ^ {rT}
$$

Therefore, the price is

$$
V_0 = e^{-rT}h(F) + \int_{0}^{F} h''(K)P(K) \,dK + \int_{F}^{\infty} h''(K)C(K) \,dK
$$


For SPX, from Part 2

'45': {'F': 3663.3762493669747,
  'alpha': 1.8165044322089068,
  'beta': 0.7,
  'rho': -0.4043017551686191,
  'nu': 2.790158328184338},

For SPY, from Part 2

'45': {'F': 366.11256803322914,
  'alpha': 0.9081326349323565,
  'beta': 0.7,
  'rho': -0.4887794472282475,
  'nu': 2.72851634129598},


In [52]:
SPX_alpha = 1.8165044322089068

SPX_beta = 0.7

SPX_rho = -0.4043017551686191

SPX_nu = 2.790158328184338

In [53]:
SPY_alpha = 0.9081326349323565

SPY_beta = 0.7

SPY_rho = -0.4887794472282475

SPY_nu = 2.72851634129598

In [54]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [55]:
def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesCall(S, K, r, sabr_vol, T)


def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesPut(S, K, r, sabr_vol, T)

In [56]:
def sabr_price(S, K, r, alpha, beta, rho, nu, T):
    F = S * np.exp(r * T)
    
    # Integrate the h_prime_prime multiplied by SABRPut price
    I_put = quad(
        lambda x: h_prime_prime(x) * SABRPut(S, K, r, alpha, beta, rho, nu, T),
        0,
        F,
    )
    
    # Integrate the h_prime_prime multiplied by SABRCall price
    I_call = quad(
        lambda x: h_prime_prime(x) * SABRCall(S, K, r, alpha, beta, rho, nu, T),
        F,
        np.inf,
    )
    V_0_SABR = np.exp(-r * T) * h_func(F) + I_put[0] + I_call[0]
    return V_0_SABR

In [57]:
SPX_SABR_price = sabr_price(SPX_spot, 
                            SPX_strike,
                            SPX_df["rate_decimal"][0], 
                            SPX_alpha, 
                            SPX_beta, 
                            SPX_rho, 
                            SPX_nu, 
                            SPX_df["years_to_expiry"][0])

print(f"Price of the Exotic option using the SABR model with SPX as the underlying is {SPX_SABR_price:5f}")

Price of the Exotic option using the SABR model with SPX as the underlying is 37.713597


In [58]:
SPY_SABR_price = sabr_price(SPY_spot, 
                            SPY_strike,
                            SPY_df["rate_decimal"][0], 
                            SPY_alpha, 
                            SPY_beta, 
                            SPY_rho, 
                            SPY_nu, 
                            SPY_df["years_to_expiry"][0])

print(f"Price of the Exotic option using the SABR model with SPY as the underlying is {SPY_SABR_price:5f}")

Price of the Exotic option using the SABR model with SPY as the underlying is 26.000677


# Exotic No.2 : Black Scholes

$$
E\Biggl[\int_{0}^{T} \sigma_t^2 \,dt\Biggr] = 2e^{rT}\Biggl(\int_{0}^{F} \frac{P(K)}{K^2} \,dK + \int_{F}^{\infty} \frac{C(K)}{K^2} \,dK\Biggr)
$$


In [59]:
def BlackScholescallintegrand(S, K, r, sigma, T):
    price = BlackScholesCall(S, K, r, sigma, T) / K**2
    return price


def BlackScholesputintegrand(S, K, r, sigma, T):
    price = BlackScholesPut(S, K, r, sigma, T) / K**2
    return price

In [60]:
SPX_forward = SPX_spot * np.exp(SPX_df["rate_decimal"][0] * SPX_df["years_to_expiry"][0])

SPX_call_integrated = quad(lambda x: BlackScholescallintegrand(SPX_spot,
                                                               x,
                                                               SPX_df["rate_decimal"][0],
                                                               SPX_ATM_sigma,
                                                               SPX_df["years_to_expiry"][0]), SPX_forward, np.inf)

SPX_put_integrated = quad(lambda x: BlackScholesputintegrand(SPX_spot,
                                                             x,
                                                             SPX_df["rate_decimal"][0],
                                                             SPX_ATM_sigma,
                                                             SPX_df["years_to_expiry"][0]), 0.0, SPX_forward)

SPX_BS_Expected_variance = 2 * np.exp(SPX_df["rate_decimal"][0] * SPX_df["years_to_expiry"][0]) * (SPX_put_integrated[0] + SPX_call_integrated[0])

print(f"The integrated variance for SPX using the Black Scholes model is {SPX_BS_Expected_variance:.5f}")

The integrated variance for SPX using the Black Scholes model is 0.00433


In [61]:
SPY_forward = SPY_spot * np.exp(SPY_df["rate_decimal"][0] * SPY_df["years_to_expiry"][0])

SPY_call_integrated = quad(lambda x: BlackScholescallintegrand(SPY_spot,
                                                               x,
                                                               SPY_df["rate_decimal"][0],
                                                               SPY_ATM_sigma,
                                                               SPY_df["years_to_expiry"][0]), SPY_forward, np.inf)

SPY_put_integrated = quad(lambda x: BlackScholesputintegrand(SPY_spot,
                                                             x,
                                                             SPY_df["rate_decimal"][0],
                                                             SPY_ATM_sigma,
                                                             SPY_df["years_to_expiry"][0]), 0.0, SPY_forward)

SPY_BS_Expected_variance = 2 * np.exp(SPY_df["rate_decimal"][0] * SPY_df["years_to_expiry"][0]) * (SPY_put_integrated[0] + SPY_call_integrated[0])

print(f"The integrated variance for SPY using the Black Scholes model is {SPY_BS_Expected_variance:.5f}")

The integrated variance for SPY using the Black Scholes model is 0.00478


# Exotic No.2 : Bachelier

In [62]:
def bach_call_integrand(S, K, r, sigma, T):
    bachmodel = BachelierCall(S, K, r, sigma, T)
    return bachmodel / K**2

def bach_put_integrand(S, K, r, sigma, T):
    bachmodel = BachelierPut(S, K, r, sigma, T)
    return bachmodel / K**2

In [63]:
SPX_forward = SPX_spot * np.exp(SPX_df["rate_decimal"][0] * SPX_df["years_to_expiry"][0])

SPX_call_integrated = quad(lambda x: bach_call_integrand(SPX_spot,
                                                             x,
                                                             SPX_df["rate_decimal"][0],
                                                             SPX_bachelier_sigma,
                                                             SPX_df["years_to_expiry"][0]), SPX_forward, np.inf)

SPX_put_integrated = quad(lambda x: bach_put_integrand(SPX_spot,
                                                               x,
                                                               SPX_df["rate_decimal"][0],
                                                               SPX_bachelier_sigma,
                                                               SPX_df["years_to_expiry"][0]), 0.0, SPX_forward)

SPX_Bachelier_Expected_variance = 2 * np.exp(SPX_df["rate_decimal"][0] * SPX_df["years_to_expiry"][0]) * (SPX_put_integrated[0] + SPX_call_integrated[0])

print(f"The integrated variance for SPX using the Bachelier model is {SPX_Bachelier_Expected_variance:.5f}")

The integrated variance for SPX using the Bachelier model is 0.00436


In [64]:
SPY_forward = SPY_spot * np.exp(SPY_df["rate_decimal"][0] * SPY_df["years_to_expiry"][0])

SPY_call_integrated = quad(lambda x: bach_call_integrand(SPY_spot,
                                                             x,
                                                             SPY_df["rate_decimal"][0],
                                                             SPY_bachelier_sigma,
                                                             SPY_df["years_to_expiry"][0]), SPY_forward, np.inf)

SPY_put_integrated = quad(lambda x: bach_put_integrand(SPY_spot,
                                                               x,
                                                               SPY_df["rate_decimal"][0],
                                                               SPY_bachelier_sigma,
                                                               SPY_df["years_to_expiry"][0]), 0.0, SPY_forward)

SPY_Bachelier_Expected_variance = 2 * np.exp(SPY_df["rate_decimal"][0] * SPY_df["years_to_expiry"][0]) * (SPY_put_integrated[0] + SPY_call_integrated[0])

print(f"The integrated variance for SPY using the Bachelier model is {SPY_Bachelier_Expected_variance:.5f}")

The integrated variance for SPY using the Bachelier model is 0.00481


# Exotic No.2 : SABR

In [65]:
def sabrcallintegrand(S, K, r, alpha, beta, rho, nu, T):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price


def sabrputintegrand(S, K, r, alpha, beta, rho, nu, T):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price

In [66]:
SPX_forward = SPX_spot * np.exp(SPX_df["rate_decimal"][0] * SPX_df["years_to_expiry"][0])

SPX_call_integrated = quad(lambda x: sabrcallintegrand(SPX_spot,
                                                        x,
                                                        SPX_df["rate_decimal"][0],
                                                        SPX_alpha,
                                                        SPX_beta,
                                                        SPX_rho,
                                                        SPX_nu,
                                                        SPX_df["years_to_expiry"][0]), SPX_forward, np.inf)

SPX_put_integrated = quad(lambda x: sabrputintegrand(SPX_spot,
                                                        x,
                                                        SPX_df["rate_decimal"][0],
                                                        SPX_alpha,
                                                        SPX_beta,
                                                        SPX_rho,
                                                        SPX_nu,
                                                        SPX_df["years_to_expiry"][0]), 0.0, SPX_forward)

SPX_SABR_Expected_variance = 2 * np.exp(SPX_df["rate_decimal"][0] * SPX_df["years_to_expiry"][0]) * (SPX_put_integrated[0] + SPX_call_integrated[0])

print(f"The integrated variance for SPX using the SABR model is {SPX_SABR_Expected_variance:.5f}")

The integrated variance for SPX using the SABR model is 0.00635


In [67]:
SPY_forward = SPY_spot * np.exp(SPY_df["rate_decimal"][0] * SPY_df["years_to_expiry"][0])

SPY_call_integrated = quad(lambda x: sabrcallintegrand(SPY_spot,
                                                        x,
                                                        SPY_df["rate_decimal"][0],
                                                        SPY_alpha,
                                                        SPY_beta,
                                                        SPY_rho,
                                                        SPY_nu,
                                                        SPY_df["years_to_expiry"][0]), SPY_forward, np.inf)

SPY_put_integrated = quad(lambda x: sabrputintegrand(SPY_spot,
                                                        x,
                                                        SPY_df["rate_decimal"][0],
                                                        SPY_alpha,
                                                        SPY_beta,
                                                        SPY_rho,
                                                        SPY_nu,
                                                        SPY_df["years_to_expiry"][0]), 0.0, SPY_forward)

SPY_SABR_Expected_variance = 2 * np.exp(SPY_df["rate_decimal"][0] * SPY_df["years_to_expiry"][0]) * (SPY_put_integrated[0] + SPY_call_integrated[0])

print(f"The integrated variance for SPY using the SABR model is {SPY_SABR_Expected_variance:.5f}")

The integrated variance for SPY using the SABR model is 0.00602
